# A legnagyobb sajátérték kiszámításának tesztelése
A cél annak az ellenőrzése, hogy
- mennyi ideig tart kiszámítani a legnagyobb sajátértéket,
- mennyire pontosan számolja ki az eigs függvény ha csak a legnagyobbat, vagy a legnagyobb 100-at számoljuk ki,
- az eredeti K-M párossal érdemes-e számolni, vagy a SymRCM-mel csökkentett sávszélességű Kp-Mp mátrixokkal?

In [1]:
import gmsh
gmsh.initialize()
using LinearAlgebra, SparseArrays

In [2]:
# Adatok
E = 2e5
ν = 0.3
ρ = 7.85e-9

height = 100
base = 1000
elemSize = 1
traction = 1

approxOrder = 1
internalNodes = 0
quadElements = 1

1

In [3]:
# Geometria megrajzolása
gmsh.model.add("rectangle")

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

sf1 = gmsh.model.occ.addPlaneSurface([cl1])

gmsh.model.occ.synchronize()

In [4]:
# Megfogások és terhelések helyének megadása
# az egyes peremfeltételek helyét megadó geometriai objektumokat
# csoportokba rendezzük

# megfogás 1
supp1 = gmsh.model.addPhysicalGroup(1, [l1])
gmsh.model.setPhysicalName(1, supp1, "FixBottomY")

# megfogás 2
supp2 = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1, supp2, "FixLeftX")

# terhelés
trac1 = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1, trac1, "ForceTopY")

In [5]:
# Végeselemes felosztás elvégzése

# lekérjük az összes csomópontot
all = gmsh.model.getEntities(0)
# megadjuk, hogy a csomóponthoz rendelt eleméret mekkora legyen
gmsh.model.mesh.setSize(all, elemSize)
# kiválasztjuk a 8-as számú hálózó algoritmust a 2D-s sf1 felülethez
gmsh.model.mesh.setAlgorithm(2, sf1, 8)
# legeneráljuk a hálót a felület kontúrjához (1D-s)
gmsh.model.mesh.generate(1)
# legeneráljuk a hálót a felülethez (2D-s)
gmsh.model.mesh.generate(2)
# a legenerált háromszög elemeket négyszög elemekké alakítjuk
if quadElements == 1
    gmsh.model.mesh.recombine()
end
# másodfokú elemekhez:
# belső csomópontok használata
if internalNodes == 1
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 0) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
else
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 1) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
end
# közelítés fokszáma (1-től 5-ig)
gmsh.model.mesh.setOrder(approxOrder)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00483506s, CPU 0.003411s)
Info    : 2200 nodes 2204 elements
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Done meshing 2D (Wall 30.858s, CPU 27.7398s)
Info    : 101101 nodes 202204 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 298900 internal 2196 closed
Info    : Blossom recombination completed (Wall 7.48008s, CPU 7.11058s): 100000 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.999899, min Q = 0.937024
Info    : Done recombining 2D mesh (Wall 7.54693s, CPU 7.17593s)


In [6]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 1)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [7]:
# Anyagállandók mátrixa SA feladathoz
D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0; ν 1-ν 0; 0 0 (1-2ν)/2]

3×3 Matrix{Float64}:
 2.69231e5  1.15385e5      0.0
 1.15385e5  2.69231e5      0.0
 0.0        0.0        76923.1

In [8]:
# Anyagállandók mátrixa ÁSF feladathoz
D = E / (1 - ν^2) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]

3×3 Matrix{Float64}:
     2.1978e5  65934.1           0.0
 65934.1           2.1978e5      0.0
     0.0           0.0       76923.1

In [9]:
# Merevségi mátrix felépítése
# csomópontok sorszámának lekérése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
# végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
# a lefoglalandó memória méretének becslése
lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * 2)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
nn = []
I = []
J = []
V = []
V = convert(Vector{Float64}, V)
sizehint!(I, lengthOfIJV)
sizehint!(J, lengthOfIJV)
sizehint!(V, lengthOfIJV)
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2approxOrder))
    numIntPoints = length(intWeights)
    #comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
    #h = reshape(fun, :, numIntPoints)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
    ∇h = reshape(dfun, :, numIntPoints)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    invJac = zeros(3, 3numIntPoints)
    Iidx = zeros(Int, numNodes * 2, numNodes * 2)
    Jidx = zeros(Int, numNodes * 2, numNodes * 2)
    for k in 1:numNodes*2, l in 1:numNodes*2
        Iidx[k, l] = l
        Jidx[k, l] = k
    end
    nn2 = zeros(Int, 2 * numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        for k in 1:numIntPoints
            invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
        end
        ∂h = zeros(3, numNodes * numIntPoints) # ∂h-t mindig csak felül kellene írni, nem kell újra és újra memóriát foglalni neki.
        for k in 1:numIntPoints, l in 1:numNodes
            ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
        end
        B = zeros(3 * numIntPoints, 2 * numNodes) # B-t mindig csak felül kellene írni?
        for k in 1:numIntPoints, l in 1:numNodes
            B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
            B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
        end
        K1 = zeros(2 * numNodes, 2 * numNodes)
        for k in 1:numIntPoints
            B1 = B[k*3-2:k*3, 1:2*numNodes]
            K1 += B1' * D * B1 * jacDet[k] * intWeights[k]
        end
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        append!(I, nn2[Iidx[:]])
        append!(J, nn2[Jidx[:]])
        append!(V, K1[:])
    end
    push!(nn, nnet)
end
K = sparse(I, J, V)
K0 = copy(K)
display(K)

202202×202202 SparseMatrixCSC{Float64, Int64} with 3613204 stored entries:
⎡⣿⣿⣦⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠈⢿⣿⣿⣷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠹⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣿⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣿⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣦⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣷⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣦⣀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⣿⣿⎦

In [10]:
# Tömeg mátrix felépítése
# csomópontok sorszámának lekérése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
# végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
# a lefoglalandó memória méretének becslése
lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * 2)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
nn = []
I = []
J = []
V = [] # Ezt vajon nem kellene átnevezni másnak? Ebben voltak a K elemei is... A 'sparse' parancs készített róla másolatot?
V = convert(Vector{Float64}, V)
sizehint!(I, lengthOfIJV)
sizehint!(J, lengthOfIJV)
sizehint!(V, lengthOfIJV)
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2approxOrder))
    numIntPoints = length(intWeights)
    comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
    h = reshape(fun, :, numIntPoints)
    #comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
    #∇h = reshape(dfun, :, numIntPoints)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    #invJac = zeros(3, 3numIntPoints)
    Iidx = zeros(Int, numNodes * 2, numNodes * 2)
    Jidx = zeros(Int, numNodes * 2, numNodes * 2)
    for k in 1:numNodes*2, l in 1:numNodes*2
        Iidx[k, l] = l
        Jidx[k, l] = k
    end
    nn2 = zeros(Int, 2 * numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        #Jac = reshape(jac, 3, :)
        #for k in 1:numIntPoints
        #    invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
        #end
        #∂h = zeros(3, numNodes * numIntPoints)
        #for k in 1:numIntPoints, l in 1:numNodes
        #    ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
        #end
        #B = zeros(3 * numIntPoints, 2 * numNodes)
        #for k in 1:numIntPoints, l in 1:numNodes
        #    B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
        #    B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
        #end
        H = zeros(2 * numIntPoints, 2 * numNodes)
        for k in 1:numIntPoints, l in 1:numNodes
            H[k*2-1, l*2-1] = H[k*2-0, l*2-0] = h[(k-1)*numNodes+l]
        end
        M1 = zeros(2 * numNodes, 2 * numNodes)
        for k in 1:numIntPoints
            H1 = H[k*2-1:k*2, 1:2*numNodes]
            M1 += H1' * H1 * jacDet[k] * intWeights[k]
        end
        M1 *= ρ
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        append!(I, nn2[Iidx[:]])
        append!(J, nn2[Jidx[:]])
        append!(V, M1[:])
    end
    push!(nn, nnet)
end
M = sparse(I, J, V)
M0 = copy(M)
display(M)

202202×202202 SparseMatrixCSC{Float64, Int64} with 3613204 stored entries:
⎡⣿⣿⣦⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠈⢿⣿⣿⣷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠹⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣿⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣿⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣦⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣧⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣷⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣦⣀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⣿⣿⎦

In [11]:
using SymRCM
using Arpack
using BenchmarkTools
perm = symrcm(K)
Kp = K[perm, perm]
Mp = M[perm, perm]
display(Kp)
λ²₁, ϕ₁ = Arpack.eigs(K, M; nev=1, which=:LM)
λ²₂, ϕ₂ = Arpack.eigs(Kp, Mp; nev=1, which=:LM)
λ²₃, ϕ₃ = Arpack.eigs(K, M; nev=100, which=:LM)
λ²₄, ϕ₄ = Arpack.eigs(Kp, Mp; nev=100, which=:LM)

display(λ²₁[1])
display(λ²₂[1])
display(λ²₃[1])
display(λ²₄[1])

display(norm(K * ϕ₁ - λ²₁[1] * M * ϕ₁))
display(norm(Kp * ϕ₂ - λ²₂[1] * Mp * ϕ₂))
display(norm(K * ϕ₃[:, 1] - λ²₃[1] * M * ϕ₃[:, 1]))
display(norm(Kp * ϕ₄[:, 1] - λ²₄[1] * Mp * ϕ₄[:, 1]))

202202×202202 SparseMatrixCSC{Float64, Int64} with 3613204 stored entries:
⎡⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

3.360813795231074e14 + 0.0im

3.360813795231057e14 + 0.0im

3.360813795231008e14 + 0.0im

3.3608137952309975e14 + 0.0im

1.5710217886451587e-9

3.1410485544442077e-9

1.1479443631784983e-8

1.3755873403643362e-8

In [12]:
using SymRCM
using Arpack
perm = symrcm(K)
Kp = K[perm, perm]
Mp = M[perm, perm]
λ², ϕ = Arpack.eigs(Kp, Mp; nev=1, which=:LM)

display(λ²[1])

display(norm(Kp * ϕ - λ²[1] * Mp * ϕ))

3.36081379523107e14 + 0.0im

1.5068902712006772e-9

In [13]:
@benchmark Arpack.eigs(K, M; nev=1, which=:LM)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 42.160 s (0.03% GC) to evaluate,
 with a memory estimate of 1.99 GiB, over 6271 allocations.

In [14]:
@benchmark Arpack.eigs(Kp, Mp; nev=1, which=:LM)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 40.393 s (0.02% GC) to evaluate,
 with a memory estimate of 2.03 GiB, over 6570 allocations.

In [29]:
gmsh.finalize()